In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [105]:
batch_size = 5  # Batch size for training.
epochs =700  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 100  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "seq2seq.txt"

In [106]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, "r", encoding="utf-8") as f:
    
    lines = f.read().split("\n")
for line in lines[1:141]:
    input_text, target_text = line.split("\t")
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


Number of samples: 140
Number of unique input tokens: 75
Number of unique output tokens: 62
Max sequence length for inputs: 204
Max sequence length for outputs: 68


In [107]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [108]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")

Epoch 1/700
23/23 [==============================] - 14s 387ms/step - loss: 2.0229 - accuracy: 0.5797 - val_loss: 1.6251 - val_accuracy: 0.5940
Epoch 2/700
23/23 [==============================] - 8s 351ms/step - loss: 1.5565 - accuracy: 0.6088 - val_loss: 1.6074 - val_accuracy: 0.5972
Epoch 3/700
23/23 [==============================] - 8s 355ms/step - loss: 1.7026 - accuracy: 0.5982 - val_loss: 1.7409 - val_accuracy: 0.5919
Epoch 4/700
23/23 [==============================] - 8s 353ms/step - loss: 1.5400 - accuracy: 0.6091 - val_loss: 1.5824 - val_accuracy: 0.6014
Epoch 5/700
23/23 [==============================] - 8s 365ms/step - loss: 1.4655 - accuracy: 0.6162 - val_loss: 1.5817 - val_accuracy: 0.5872
Epoch 6/700
23/23 [==============================] - 8s 359ms/step - loss: 1.4144 - accuracy: 0.6272 - val_loss: 1.4729 - val_accuracy: 0.6087
Epoch 7/700
23/23 [==============================] - 9s 377ms/step - loss: 1.3191 - accuracy: 0.6468 - val_loss: 2.2758 - val_accuracy: 0.614

Epoch 58/700
23/23 [==============================] - 9s 395ms/step - loss: 0.2274 - accuracy: 0.9380 - val_loss: 1.0284 - val_accuracy: 0.7537
Epoch 59/700
23/23 [==============================] - 8s 352ms/step - loss: 0.2170 - accuracy: 0.9407 - val_loss: 1.0592 - val_accuracy: 0.7500
Epoch 60/700
23/23 [==============================] - 9s 371ms/step - loss: 0.2097 - accuracy: 0.9437 - val_loss: 1.0608 - val_accuracy: 0.7616
Epoch 61/700
23/23 [==============================] - 8s 359ms/step - loss: 0.2065 - accuracy: 0.9447 - val_loss: 1.1020 - val_accuracy: 0.7500
Epoch 62/700
23/23 [==============================] - 8s 353ms/step - loss: 0.1989 - accuracy: 0.9463 - val_loss: 1.0531 - val_accuracy: 0.7642
Epoch 63/700
23/23 [==============================] - 9s 376ms/step - loss: 0.1909 - accuracy: 0.9484 - val_loss: 1.1543 - val_accuracy: 0.7463
Epoch 64/700
23/23 [==============================] - 8s 362ms/step - loss: 0.1857 - accuracy: 0.9497 - val_loss: 1.1371 - val_accuracy:

Epoch 115/700
23/23 [==============================] - 8s 365ms/step - loss: 0.0881 - accuracy: 0.9711 - val_loss: 1.4514 - val_accuracy: 0.7542
Epoch 116/700
23/23 [==============================] - 8s 364ms/step - loss: 0.0872 - accuracy: 0.9707 - val_loss: 1.4153 - val_accuracy: 0.7521
Epoch 117/700
23/23 [==============================] - 8s 351ms/step - loss: 0.0867 - accuracy: 0.9710 - val_loss: 1.4268 - val_accuracy: 0.7547
Epoch 118/700
23/23 [==============================] - 8s 346ms/step - loss: 0.0878 - accuracy: 0.9710 - val_loss: 1.4420 - val_accuracy: 0.7505
Epoch 119/700
23/23 [==============================] - 8s 356ms/step - loss: 0.0870 - accuracy: 0.9714 - val_loss: 1.4488 - val_accuracy: 0.7474
Epoch 120/700
23/23 [==============================] - 8s 353ms/step - loss: 0.0861 - accuracy: 0.9718 - val_loss: 1.4355 - val_accuracy: 0.7521
Epoch 121/700
23/23 [==============================] - 8s 349ms/step - loss: 0.0865 - accuracy: 0.9703 - val_loss: 1.4561 - val_ac

23/23 [==============================] - 8s 354ms/step - loss: 0.0841 - accuracy: 0.9718 - val_loss: 1.6234 - val_accuracy: 0.7453
Epoch 172/700
23/23 [==============================] - 8s 357ms/step - loss: 0.0791 - accuracy: 0.9728 - val_loss: 1.5914 - val_accuracy: 0.7411
Epoch 173/700
23/23 [==============================] - 8s 355ms/step - loss: 0.0768 - accuracy: 0.9727 - val_loss: 1.5946 - val_accuracy: 0.7390
Epoch 174/700
23/23 [==============================] - 8s 358ms/step - loss: 0.0786 - accuracy: 0.9720 - val_loss: 1.5982 - val_accuracy: 0.7521
Epoch 175/700
23/23 [==============================] - 8s 343ms/step - loss: 0.0865 - accuracy: 0.9694 - val_loss: 1.5739 - val_accuracy: 0.7374
Epoch 176/700
23/23 [==============================] - 8s 339ms/step - loss: 0.0839 - accuracy: 0.9710 - val_loss: 1.5655 - val_accuracy: 0.7416
Epoch 177/700
23/23 [==============================] - 8s 351ms/step - loss: 0.0787 - accuracy: 0.9718 - val_loss: 1.6473 - val_accuracy: 0.7337

23/23 [==============================] - 8s 342ms/step - loss: 0.0732 - accuracy: 0.9732 - val_loss: 1.6275 - val_accuracy: 0.7468
Epoch 228/700
23/23 [==============================] - 8s 341ms/step - loss: 0.0784 - accuracy: 0.9716 - val_loss: 1.6449 - val_accuracy: 0.7426
Epoch 229/700
23/23 [==============================] - 8s 347ms/step - loss: 0.0736 - accuracy: 0.9737 - val_loss: 1.6392 - val_accuracy: 0.7511
Epoch 230/700
23/23 [==============================] - 8s 366ms/step - loss: 0.0744 - accuracy: 0.9735 - val_loss: 1.6466 - val_accuracy: 0.7453
Epoch 231/700
23/23 [==============================] - 8s 356ms/step - loss: 0.0740 - accuracy: 0.9736 - val_loss: 1.6912 - val_accuracy: 0.7437
Epoch 232/700
23/23 [==============================] - 8s 347ms/step - loss: 0.1295 - accuracy: 0.9602 - val_loss: 1.5729 - val_accuracy: 0.7432
Epoch 233/700
23/23 [==============================] - 8s 351ms/step - loss: 0.1011 - accuracy: 0.9664 - val_loss: 1.6466 - val_accuracy: 0.7437

...

23/23 [==============================] - 8s 343ms/step - loss: 0.0494 - accuracy: 0.9821 - val_loss: 1.9998 - val_accuracy: 0.7442
Epoch 564/700
23/23 [==============================] - 8s 344ms/step - loss: 0.0430 - accuracy: 0.9842 - val_loss: 2.0056 - val_accuracy: 0.7437
Epoch 565/700
23/23 [==============================] - 8s 340ms/step - loss: 0.0441 - accuracy: 0.9841 - val_loss: 2.0091 - val_accuracy: 0.7484
Epoch 566/700
23/23 [==============================] - 8s 339ms/step - loss: 0.0433 - accuracy: 0.9833 - val_loss: 2.0329 - val_accuracy: 0.7500
Epoch 567/700
23/23 [==============================] - 7s 322ms/step - loss: 0.0431 - accuracy: 0.9832 - val_loss: 2.0260 - val_accuracy: 0.7405
Epoch 568/700
23/23 [==============================] - 8s 333ms/step - loss: 0.0436 - accuracy: 0.9837 - val_loss: 2.0185 - val_accuracy: 0.7474
Epoch 569/700
23/23 [==============================] - 8s 369ms/step - loss: 0.0445 - accuracy: 0.9838 - val_loss: 2.0266 - val_accuracy: 0.7511

23/23 [==============================] - 8s 359ms/step - loss: 0.0506 - accuracy: 0.9802 - val_loss: 1.8741 - val_accuracy: 0.7610
Epoch 620/700
23/23 [==============================] - 8s 353ms/step - loss: 0.0484 - accuracy: 0.9807 - val_loss: 1.9167 - val_accuracy: 0.7563
Epoch 621/700
23/23 [==============================] - 8s 346ms/step - loss: 0.0487 - accuracy: 0.9815 - val_loss: 1.9421 - val_accuracy: 0.7547
Epoch 622/700
23/23 [==============================] - 8s 358ms/step - loss: 0.0492 - accuracy: 0.9803 - val_loss: 1.9174 - val_accuracy: 0.7516
Epoch 623/700
23/23 [==============================] - 8s 337ms/step - loss: 0.0464 - accuracy: 0.9812 - val_loss: 1.9443 - val_accuracy: 0.7547
Epoch 624/700
23/23 [==============================] - 8s 354ms/step - loss: 0.0481 - accuracy: 0.9820 - val_loss: 1.9510 - val_accuracy: 0.7547
Epoch 625/700
23/23 [==============================] - 8s 347ms/step - loss: 0.0474 - accuracy: 0.9825 - val_loss: 1.9649 - val_accuracy: 0.7468

23/23 [==============================] - 8s 344ms/step - loss: 0.0425 - accuracy: 0.9838 - val_loss: 2.0781 - val_accuracy: 0.7453
Epoch 676/700
23/23 [==============================] - 8s 348ms/step - loss: 0.0464 - accuracy: 0.9829 - val_loss: 2.1183 - val_accuracy: 0.7437
Epoch 677/700
23/23 [==============================] - 8s 355ms/step - loss: 0.0450 - accuracy: 0.9833 - val_loss: 2.1055 - val_accuracy: 0.7474
Epoch 678/700
23/23 [==============================] - 8s 349ms/step - loss: 0.0441 - accuracy: 0.9835 - val_loss: 2.1512 - val_accuracy: 0.7395
Epoch 679/700
23/23 [==============================] - 8s 354ms/step - loss: 0.0445 - accuracy: 0.9833 - val_loss: 2.1353 - val_accuracy: 0.7458
Epoch 680/700
23/23 [==============================] - 8s 354ms/step - loss: 0.0439 - accuracy: 0.9837 - val_loss: 2.1624 - val_accuracy: 0.7421
Epoch 681/700
23/23 [==============================] - 8s 341ms/step - loss: 0.0471 - accuracy: 0.9827 - val_loss: 2.1417 - val_accuracy: 0.7405

INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


In [99]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [102]:
for seq_index in range(50):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)
    
    
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Abbotsford Regional Hospital and Cancer Centre Fraser Health Authority Suite 400 Central City Tower 13450 - 102 Avenue Surrey BC  V3T 0H1
Decoded sentence: South Okanagan General Hospital

-
Input sentence: Abbotsford Regional Hospital and Cancer Centre Fraser Health Authority Suite 400 Central City Tower 13450 - 102 Avenue Surrey BC  V3T 0H1
Decoded sentence: South Okanagan General Hospital

-
Input sentence: Cormorant Island Community Health Centre Vancouver Island Health Authority 1952 Bay Street Victoria, BC, V8R 1J8
Decoded sentence: St. John Hospital

-
Input sentence: Cormorant Island Community Health Centre Vancouver Island Health Authority 1952 Bay Street Victoria, BC, V8R 1J8
Decoded sentence: St. John Hospital

-
Input sentence: Ashcroft and District General Hospital Interior Health Authority Community Health and Services Centre 5th Floor 505 Doyle Avenue Kelowna, BC, V1Y 6V8
Decoded sentence: St. Paul's Hospital

-
Input sentence: Ashcroft and District Gen

KeyboardInterrupt: 

In [70]:
encoder_input_data[1]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [103]:
new_input = "North York General Hospital Fraser Health Authority, 194 King Street East Toronto ON V1K 4H5"

new_encoder_input_data = np.zeros(
    (1,max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)

for t, char in enumerate(new_input):
    new_encoder_input_data[0,t, input_token_index[char]] = 1.0

In [104]:
decode_sequence(new_encoder_input_data)

'Port McNeill and District Hospital\n'

88